## RNN의 입출력 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현한다면, 이를 문자 단위 RNN이라고 한다. 다대다 구조로 구현해보자. 

### 1. 문자 단위 RNN(Char RNN)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#### 1) 훈련 데이터 전처리하기

입력 데이터와 레이블 데이터에 대해서 문자 집합(vocabulary)를 만든다.  
여기서 문자 집합은 중복을 제거한 문자들의 집합이다.  

In [21]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print(f'문자 집합의 크기 : {vocab_size}')

문자 집합의 크기 : 5


문자 집합의 문자는 총 5개이다. '!,a,e,l,p'.

이제 하이퍼파라미터를 정의한다.  
입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야만 한다.

In [22]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

문자 집합에 고유한 정수를 부여한다. 

In [23]:
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
# 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


나중에 예측 결과를 다시 문자 시퀀스로 보기 위해서   
반대로 정수로부터 문자를 얻을 수 있는 index_to_char를 만든다.

In [24]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


이제 입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑한다.

In [25]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받는다.  
따라서 배치 차원을 추가해준다.

In [26]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔준다.

In [27]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


입력 데이터와 레이블 데이터를 텐서로 바꿔준다.

In [28]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

이제 각 텐서의 크기를 확인해보자.

In [29]:
print(f'훈련 데이터의 크기 : {X.shape}')
print(f'레이블의 크기 : {Y.shape}')

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


#### 2) 모델 구현하기

이제 RNN 모델을 구현하자.  
fc는 fully-connected layer를 뜻하며, 출력층으로 사용된다.

In [30]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size,hidden_size,batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

클래스로 정의한 모델을 net에 저장한다.

In [31]:
net = Net(input_size, hidden_size, output_size)

이제 입력된 모델에 입력을 넣어서 출력의 크기를 확인해보자.

In [32]:
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


(1,5,5)의 크기를 갖는데, 각각  
배치 차원,  
시점,  
출력의 크기   
이다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데,  
이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만든다. 

In [33]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


이제 레이블 데이터의 크기를 리마인드 해보자.

In [34]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


레이블 데이터는 (1,5)의 크기를 가지는데,   
마찬가지로 나중에 정확도를 측정할 때는 이를 펼쳐서 계산할 예정이다.   
이 경우 (5)의 크기를 가지게 된다.  

이제 옵티마이저와 손실 함수를 정의한다. 

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),learning_rate)

In [37]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    # view를 하는 이유는 Batch 차원 제거를 위함.
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하는 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택

    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])

    print(i, f'loss : {loss.item()}\nprediction : {result}\ntrue Y : {y_data}\n prediction str : {result_str}')

0 loss : 0.00020904606208205223
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
1 loss : 0.0002084262960124761
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
2 loss : 0.0002078065590467304
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
3 loss : 0.0002072344796033576
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
4 loss : 0.00020663856412284076
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
5 loss : 0.00020604263409040868
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
6 loss : 0.0002053990465356037
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
7 loss : 0.00020485080312937498
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
8 loss : 0.00020427870913408697
prediction : [[4 4 3 2 0]]
true Y : [[4, 4, 3, 2, 0]]
 prediction str : pple!
9 loss : 0.000

### 2. 더 많은 데이터로 학습한 문자 단위 RNN(Char RNN)

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

#### 1) 훈련 데이터 전처리하기

In [39]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")


문자 집합을 생성하고, 각 문자에 고유한 정수를 부여한다.

In [40]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c : i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [41]:
print(char_dic)

{',': 0, 'i': 1, 'u': 2, 's': 3, 'm': 4, 'r': 5, 'd': 6, 'p': 7, 'o': 8, ' ': 9, 'w': 10, 't': 11, '.': 12, 'f': 13, 'n': 14, "'": 15, 'h': 16, 'y': 17, 'e': 18, 'g': 19, 'b': 20, 'k': 21, 'a': 22, 'c': 23, 'l': 24}


각 문자에 정수가 부여되었으며, 총 25개의 문자가 존재한다.  
문자 집합의 크기를 확인해보자.  

In [42]:
dic_size = len(char_dic)
print(f'문자 집합의 크기 : {dic_size}')

문자 집합의 크기 : 25


문자 집합의 크기는 25이며,  
입력은 원-핫 벡터로 사용할 것이므로 매 시점마다 들어갈 입력의 크기이기도 하다.

이제 하이퍼파라미터를 설정하자.  
hidden_size(은닉 상태의 크기)를 입력의 크기와 동일하게 줬는데,  
이는 사용자의 선택으로 다른 값을 줘도 무방하다.  

그리고 sequence_length라는 변수를 선언했는데,  
우리가 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정이기 때문이다.

In [43]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10   # 임의 숫자 지정
learning_rate = 0.1

다음은 임의로 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만든다.

In [44]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36